In [ ]:
import os, sys
import numpy as np

import matplotlib.pyplot as plt
import h5py
import copy
import random


import dicom_to_numpy as dtn
from deepdrr import projector
import projection_matrix
from analytic_generators import add_noise
import mass_attenuation_gpu as mass_attenuation
import spectrum_generator
import add_scatter
from utils import Camera
import segmentation
import get_tumor_center as gtc

import index_tracker as track   

from generate_drrs import generate_drrs

**1. Set the hyperparameters for generating DRRs**

In [ ]:
# SET HYPERPARAMETERS   
# set max shifting for soft-tissue/ bone decorrelation and the number of shifts
max_x = 32
max_y = 32
nr_shifts = 9
standard = True
decorrelate = True

# set the height, width, pixel_size, the distance between the source and detector panel
# and the distance between the source and pCT (isocenter_distance) 
camera = Camera(sensor_width = int(1024) + 2*max_y, sensor_height = int(768) + 2*max_x, 
                pixel_size = 0.388, source_to_detector_distance = 1500, 
                isocenter_distance = 1000)

# set the angle perspectives and the number of simulated photons
min_theta = 270
max_theta = 271
min_phi = 90
max_phi = 271
spacing_theta = 30
spacing_phi = 1
photon_count = 500000
spectrum = spectrum_generator.SPECTRUM120KV_AL43

**2. Load the pCT dataset**

In [ ]:
data_path = ''
patient_name = 'patient1'
    
# get voxel size
filename = data_path + patient_name + '/original_data/phase_0'
with h5py.File(filename, 'r') as f:
    voxel_size = np.array(f['spacing'], dtype = 'float32')
        

# get tumor center
labs = dtn.load_data(data_path + patient_name + '/original_data/' + 'labs.hdf5', 'labs')    
labs = np.flip(labs, axis = 3)    

x, y, z = gtc.get_patient_relative_center(labs)
origin = [x, y, z] * voxel_size

**3. Generate DRRs**

In [ ]:
# generate drrs
generate_patient_drrs(data_path, patient_name, voxel_size, min_theta, max_theta, min_phi, 
                          max_phi, spacing_theta, spacing_phi, photon_count, camera, spectrum, 
                          scatter = False, noise = True,
                          standard = standard, decorrelate = decorrelate,
                          max_x = max_x, max_y = max_y, nr_shifts = nr_shifts, origin = origin)